In [1]:
# Taken from:
# https://github.com/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/Stock_NeurIPS2018/Stock_NeurIPS2018_2_Train.ipynb

In [2]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-phflbg53
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-phflbg53
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 0496601ee2824ca675beb5a220afd9109ac02ffa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-dpwudzrw/elegantrl_743ca43ae73e484f8910fa492db24107
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-dpwudzrw/elegantrl_743ca43ae73e484f8910fa492db24107
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 5bca7aeeb73f5779788dd6714a53b59ce880c583
  Preparing metadata (setup.py) ... done


In [1]:
import os
import pandas as pd

from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl import config_tickers
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR

check_and_make_directories([TRAINED_MODEL_DIR])

2024-11-21 09:33:42.388867: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 09:33:42.411565: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732156422.435948  176454 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732156422.443269  176454 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 09:33:42.477374: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
train = pd.read_csv('data/train_data.csv')
# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following two lines.
# train = train.set_index(train.columns[0])
# train.index.names = ['']

In [3]:
train.columns

Index(['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd',
       'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma',
       'close_60_sma', 'vix', 'turbulence'],
      dtype='object')

In [4]:
y = sum((train[t].values.tolist() for t in INDICATORS), [])

In [5]:
len(y)

671176

In [6]:
x = train["close"].values.tolist()

In [7]:
x[0]

3.067142963409424

In [8]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 291


In [9]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "print_verbosity":5

}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [10]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

ValueError: could not broadcast input array from shape (39,) into shape (291,)

In [11]:
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

In [12]:
if_using_a2c = True

In [14]:
TRAINED_MODEL_DIR = "/home/devmiftahul/trained_models"

In [15]:
TRAINED_MODEL_DIR

'/home/devmiftahul/trained_models'

In [17]:
trained_a2c = A2C.load(TRAINED_MODEL_DIR + "/agent_a2c") if if_using_a2c else None

/home/devmiftahul/.pyenv/versions/3.10.5/envs/finrl/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [19]:
INDICATORS = ['macd',
               'rsi_30',
               'cci_30',
               'dx_30']

In [20]:
INDICATORS

['macd', 'rsi_30', 'cci_30', 'dx_30']

In [22]:
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.config_tickers import DOW_30_TICKER

TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TEST_START_DATE = '2021-10-01'
TEST_END_DATE = '2023-03-01'

df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (97013, 8)


In [24]:
print(df)

             date        open        high         low       close     volume  \
0      2010-01-04    6.447413    7.643214    7.660714    7.622500  493729600   
1      2010-01-04   40.591286   57.720001   57.869999   56.630001    5277400   
2      2010-01-04   32.906178   40.919998   41.099998   40.810001    6894300   
3      2010-01-04   43.777554   56.180000   56.389999   55.720001    6186700   
4      2010-01-04   39.883911   58.549999   59.189999   57.650002    7325600   
...           ...         ...         ...         ...         ...        ...   
97008  2023-02-28  463.423553  475.940002  483.359985  482.670013    3902100   
97009  2023-02-28  216.976074  219.940002  221.770004  220.000000    5385400   
97010  2023-02-28   34.428703   38.810001   38.970001   38.700001   16685300   
97011  2023-02-28   31.044863   35.529999   35.779999   35.480000    8847000   
97012  2023-02-28   46.214359   47.376667   47.549999   47.000000   18054000   

        tic  day  
0      AAPL    0  
1

In [26]:
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
import numpy as np
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

Successfully added technical indicators
Successfully added turbulence index


In [27]:
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 175


In [28]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [30]:
e_trade_gym = StockTradingEnv(df = processed, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

ValueError: could not broadcast input array from shape (35,) into shape (175,)